In [108]:
%matplotlib widget
%load_ext autoreload
%autoreload 2
from src.cort_processor import *
import random as rand
from src.plotter import *
from src.filters import *
import math
import pickle
import scipy
import matplotlib.pyplot as plt
from  matplotlib.colors import LinearSegmentedColormap
from matplotlib.pyplot import cm
from scipy import signal
import numpy as np
import seaborn as sns
import pandas as pd
import scipy.stats as stats
from matplotlib import colors as mcolors
from mpl_toolkits.mplot3d import Axes3D
import tdt 
import matplotlib.pyplot as plt
from scipy.signal import butter, lfilter, iirnotch, filtfilt, resample, hilbert, welch, resample
from scipy.fftpack import fft,fftfreq,rfft,irfft,ifft
from scipy.io import savemat
from src.phase_decoder_support import *
from src.neural_analysis import *
from src.wiener_filter import *
from src.folder_handler import *
from src.tdt_support import *
from src.decoders import *
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score, GridSearchCV, train_test_split
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression
from collections import Counter
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score, confusion_matrix,roc_curve, roc_auc_score, precision_score, recall_score, precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [395]:
def to_categorical(y, num_classes=None, dtype="float32"):
    y = np.array(y, dtype="int")
    input_shape = y.shape
    if input_shape and input_shape[-1] == 1 and len(input_shape) > 1:
        input_shape = tuple(input_shape[:-1])
    y = y.ravel()
    if not num_classes:
        num_classes = np.max(y) + 1
    n = y.shape[0]
    categorical = np.zeros((n, num_classes), dtype=dtype)
    categorical[np.arange(n), y] = 1
    output_shape = input_shape + (num_classes,)
    categorical = np.reshape(categorical, output_shape)

    return categorical

def index_balancer(y):
    value_counts = []
    resampled_y = []
    original_indicies = []
    for i in range(y.shape[1]):
        column_sum = np.sum(y[:,i])
        value_counts.append(column_sum)
    for i in range(y.shape[1]):
        floor = np.min(value_counts)
        extant = value_counts[i]
        likelihood = (extant-floor)/extant
        for j in range(y.shape[0]):
            if y[j,i] == 1:
                if rand.random() >= likelihood:
                    resampled_y.append(y[j, :])
                    original_indicies.append(j)
    return np.array(original_indicies)

In [381]:
meepo1004 = CortProcessor('/mnt/c/oobootoo/rat-fes/data/1004_test')
meepo1004.process(clear_storage=False)

read from t=0s to t=246.43s


KeyboardInterrupt: 

In [1]:
h_sin, h_cos, r2, predicted_arctans, test_arctans, test_rates, phase_list = meepo1004.decode_phase(metric_angle="knee")

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed')).History will not be written to the database.


NameError: name 'meepo1004' is not defined

In [383]:
phase_data2 = pd.read_csv('/mnt/c/oobootoo/rat-fes/data/1004_test/phase/1004_154010_phase.csv', header=0)
phase_data1 = pd.read_csv('/mnt/c/oobootoo/rat-fes/data/1004_test/phase/1004_153140_phase.csv', header=0)

In [384]:
time00 = phase_data1[phase_data1['fnum']==meepo1004.kin_data[0]['fnum'][0]].index.values
time01 = phase_data1[phase_data1['fnum']==meepo1004.kin_data[0]['fnum'][-1]].index.values
time10 = phase_data2[phase_data2['fnum']==meepo1004.kin_data[1]['fnum'][0]].index.values
time11 = phase_data2[phase_data2['fnum']==meepo1004.kin_data[1]['fnum'][-1]].index.values

In [385]:
meepo1004.kin_data[0]['fnum'][0]

3800

In [386]:
meepo1004.kin_data[1]['fnum'][-1]

55999

In [387]:
print(time00, time01,time10, time11)

[498] [41297] [949] [54948]


In [388]:
meepo1004.data["rates"][0].shape

(4083, 32)

In [389]:
phase1 = np.array(phase_data1.iloc[time00[0]:time01[0],-1:])
phase2 = np.array(phase_data2.iloc[time10[0]:time11[0],-1:])

[[0]
 [0]
 [0]
 ...
 [1]
 [1]
 [1]]
[[1]
 [1]
 [1]
 ...
 [1]
 [1]
 [1]]


In [390]:
resampled_phase1 = (np.rint(resample(phase1, meepo1004.data["rates"][0].shape[0], axis=0))).astype(int)
resampled_phase2 = (np.rint(resample(phase2, meepo1004.data["rates"][1].shape[0], axis=0))).astype(int)

In [391]:
binned_phase = []
binned_phase.append(resampled_phase1)
binned_phase.append(resampled_phase2)

print(binned_phase[0].shape)
print(meepo1004.data["rates"][0].shape)
print(len(binned_phase))
print(len(meepo1004.data["rates"]))

(4083, 1)
(4083, 32)
2
2


In [392]:
print(Counter(formatted_phase))
def elim_swing(X, y):
    non2ind = []
    for i in range(y.shape[0]):
        if y[i] != 2:
            non2ind.append(i)
    y_new = y[non2ind]
    X_new = X[non2ind]
    return X_new, y_new



Counter({1: 5077, 0: 2780, 2: 1610})


In [421]:
formatted_rates, formatted_phase = meepo1004.stitch_and_format(meepo1004.data["rates"],binned_phase, N=10)

In [428]:

y = formatted_phase
X = formatted_rates
for i in range(y.shape[0]):
    if y[i] == 2:
        y[i] = 1



# roll = 5
# yf_tmp = np.roll(formatted_phase, roll)[roll:]
# Xf_tmp = formatted_rates[roll:,:]
# Xf, yf = elim_swing(Xf_tmp, yf_tmp)
# f_cat = to_categorical(yf)
# f_ind = index_balancer(f_cat)
# y_forward = yf[f_ind]
# X_forward = Xf[f_ind, :]

# yb_tmp = np.roll(formatted_phase, -roll)[:-roll]
# Xb_tmp = formatted_rates[:-roll,:]
# Xb, yb = elim_swing(Xb_tmp, yb_tmp)
# b_cat = to_categorical(yb)
# b_ind = index_balancer(b_cat)
# y_backward = yb[b_ind]
# X_backward = Xb[b_ind, :]


In [429]:
w = [{0:1000,1:100},{0:1000,1:10}, {0:1000,1:1.0}, 
     {0:500,1:1.0}, {0:400,1:1.0}, {0:300,1:1.0}, {0:200,1:1.0}, 
     {0:150,1:1.0}, {0:100,1:1.0}, {0:99,1:1.0}, {0:10,1:1.0}, 
     {0:1.0,1:0.1}, {0:10,1:0.1}, {0:100,1:0.1}, 
     {0:10,1:0.01}, {0:1.0,1:0.01}, {0:1.0,1:0.001}, {0:1.0,1:0.005},  ]
hyperparam_grid = {"class_weight": w }
# define model
lg3 = LogisticRegression(random_state=13, max_iter=1000)
# define evaluation procedure
grid = GridSearchCV(lg3,hyperparam_grid,scoring="roc_auc", cv=20, n_jobs=-1, refit=True)
grid.fit(X,y)
print(f'Best score: {grid.best_score_} with param: {grid.best_params_}')

In [430]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
# X_trainf, X_testf, y_trainf, y_testf = train_test_split(X_forward, y_forward, test_size = 0.2, random_state = 42)
# X_trainb, X_testb, y_trainb, y_testb = train_test_split(X_backward, y_backward, test_size = 0.2, random_state = 42)

In [431]:
lg4 = LogisticRegression(random_state=13,max_iter= 1000)
# fit it
lg4.fit(X_train,y_train)
# test
y_pred = lg4.predict(X_test)
# performance
print(f'Accuracy Score: {accuracy_score(y_test,y_pred)}')
print(f'Confusion Matrix: \n{confusion_matrix(y_test, y_pred)}')
print(f'Area Under Curve: {roc_auc_score(y_test, y_pred)}') # 0.5
print(f'Recall score: {recall_score(y_test,y_pred)}')

Accuracy Score: 0.657942238267148
Confusion Matrix: 
[[376 197]
 [182 353]]
Area Under Curve: 0.6580042732951672
Recall score: 0.6598130841121496


In [432]:
lg4.fit(X_trainf,y_trainf)
y_predf = lg4.predict(X_testf)
print(f'Accuracy Score: {accuracy_score(y_testf,y_predf)}')
print(f'Confusion Matrix: \n{confusion_matrix(y_testf, y_predf)}')
print(f'Area Under Curve: {roc_auc_score(y_testf, y_predf)}') # 0.5
print(f'Recall score: {recall_score(y_testf,y_predf)}')

Accuracy Score: 0.6429207479964381
Confusion Matrix: 
[[352 216]
 [185 370]]
Area Under Curve: 0.6431924882629108
Recall score: 0.6666666666666666


In [433]:
lg4.fit(X_trainb,y_trainb)
y_predb = lg4.predict(X_testb)
print(f'Accuracy Score: {accuracy_score(y_testb,y_predb)}')
print(f'Confusion Matrix: \n{confusion_matrix(y_testb, y_predb)}')
print(f'Area Under Curve: {roc_auc_score(y_testb, y_predb)}') # 0.5
print(f'Recall score: {recall_score(y_testb,y_predb)}')

Accuracy Score: 0.5965703971119134
Confusion Matrix: 
[[333 240]
 [207 328]]
Area Under Curve: 0.5971179723051329
Recall score: 0.6130841121495327
